# INFO 2950 Project: Appendix 

## A: Data Cleaning

In [1]:
# Load the libraries we will need to perform data cleaning.
import numpy as np
import pandas as pd

In [2]:
# Concatenating PBP scoring files
pbp_sc_files = []
for i in np.arange(1,22):
    name = 'pbpstats_export-{}.csv'.format(i)
    file = pd.read_csv(name)
    file['Season'] = 2022-i
    pbp_sc_files.append(file)
pbp_sc = pd.concat(pbp_sc_files)

# Final PBP scoring csv
pbp_sc.to_csv('pbp_sc.csv')

In [3]:
# Concatenating PBP assist files
pbp_as_files = []
for i in np.arange(-42,-21):
    name = 'pbpstats_export-{}.csv'.format(-i)
    file = pd.read_csv(name)
    file['Season'] = 2000-21-i
    pbp_as_files.append(file)    
pbp_as = pd.concat(pbp_as_files)

# Final PBP assist csv
pbp_as.to_csv('pbp_as.csv')

In the above two cells, we combined the individual season PBP scoring and assist, respectively, data CSVs into a single CSV containing the data for every season.

In [4]:
# Reading in csv files
pbp_score = pd.read_csv('pbp_sc.csv')
pbp_assist = pd.read_csv('pbp_as.csv')
nba_pos = pd.read_csv('NBA Players.csv')
seasonStats1950to2017 = pd.read_csv('Season_Stats_1950to2017.csv')
seasonStats2017to2019 = pd.read_csv('Season_Stats_2017to2019.csv')
seasonStats2019to2021 = pd.read_csv('Season_Stats_2019to2021.csv')

Prior to reading in the 2017-2019 and 2019-2021 season data, we had to copy and paste the individual, "raw" data in separate excel sheets into a single sheet for each of those time periods. This is because when data is downloaded from Stahead as a CSV, only 100 entries can be downloaded at a time.

In [5]:
# Drop data from before 2000-2001 season
seasonStats2000to2017 = seasonStats1950to2017[seasonStats1950to2017['Year'] >= 2001]
# Reset index to default so idx doesn't start at some random value
seasonStats2000to2017 = seasonStats2000to2017.reset_index()
# Drop some irrelevant columns
seasonStats2000to2017 = seasonStats2000to2017.drop(['index', 'Unnamed: 0', 'blanl', 'blank2', 'Pos'], axis = 1)
# Change the datatype of year (= season) from float64 to int
seasonStats2000to2017 = seasonStats2000to2017.astype({'Year': int})

Due to the PBP data only going back to the 2000-2001 season, we are limiting the scope of our analysis to just the last 20 NBA seasons. So, in the above cell, we dropped all data from before the 2000-2001 season in our Basketball Reference DataFrame. Additionally, we dropped some unnecessary columns. Finally, we changed the type of the 'Year' values from float64 to int, which is the true type of a year.

In [6]:
# Add missing columns into 2019 to 2021 data
seasonStats2019to2021['FG%'] = round(seasonStats2019to2021['FG']/seasonStats2019to2021['FGA'], 3)
seasonStats2019to2021['2P%'] = round(seasonStats2019to2021['2P']/seasonStats2019to2021['2PA'],3)
seasonStats2019to2021['3P%'] = round(seasonStats2019to2021['3P']/seasonStats2019to2021['3PA'],3)
seasonStats2019to2021['FT%'] = round(seasonStats2019to2021['FT']/seasonStats2019to2021['FTA'],3)
seasonStats2019to2021['eFG%'] = round(((seasonStats2019to2021['FG'] + (0.5*seasonStats2019to2021['3P']))/
                                 seasonStats2019to2021['FGA']),3)
seasonStats2019to2021['TS%'] = round(((seasonStats2019to2021['PTS'])/
                                (2*(seasonStats2019to2021['FGA'])+(0.44*seasonStats2019to2021['FTA']))),3)

In the above cell, we added some columns into our DataFrame containing the 2019-2021 Stathead data that were missing. These columns were present in our DataFrames containing the 2000-2017 and 2017-2019 data.

In [7]:
# Drop extraneous columns from 2017 to 2019 and 2019 to 2021 data
seasonStats2017to2019 = seasonStats2017to2019.drop(['ORtg','DRtg', 'Lg'], axis = 1)
seasonStats2019to2021 = seasonStats2019to2021.drop(['ORtg','DRtg', 'Lg'], axis = 1)

In the above cell, we dropped some unnecessary columns present in the 2017-2019 and 2019-2021 Stathead DataFrames.

In [8]:
# Drop all rows with NaNs from 2017 to 2019 data.
seasonStats2017to2019 = seasonStats2017to2019.dropna(how = 'all')

Here, we had to drop some trailing rows of NaNs that were present at the end of the 2017-2019 Stathead DataFrame.

In [9]:
# Renaming and reordering columns across the 3 dfs
seasonStats2000to2017 = seasonStats2000to2017.rename(columns = {'Year': 'Season'})
cols = list(seasonStats2000to2017.columns)
seasonStats2017to2019 = seasonStats2017to2019[cols]
seasonStats2019to2021 = seasonStats2019to2021[cols]

In the above cell, we had to rename the 'Year' column in the 2000-2017 Dataframe to match the corresponding column title in the 2017-2019 and 2019-2021 DataFrames. Additionally, we reordered the columns in the 2017-2019 and 2019-2021 DataFrames to match the other DataFrame. This will ease the process of combining the three DataFrames.

In [10]:
# Converting Season column values to be same across the 3 dfs
for i in np.arange(len(seasonStats2017to2019)):
    seasonStats2017to2019.loc[i,'Season'] = int(seasonStats2017to2019.loc[i,'Season'][-2:])+2000

for i in np.arange(len(seasonStats2019to2021)):
    seasonStats2019to2021.loc[i,'Season'] = int(seasonStats2019to2021.loc[i,'Season'][-2:])+2000

In the above cell, we converted the 'Season' column values in two of the three DataFrames so that they would match across all three. Once again, this will simplify the process of combining the three DataFrames.

In [11]:
# Combining the 3 df into a single Stathead df
seasonStats = [seasonStats2000to2017,seasonStats2017to2019,seasonStats2019to2021]
nba = pd.concat(seasonStats)

Here, we actually perform the concatenation, creating a single DataFrame corresponding to all of our Stathead/Basketball Reference data.

In [12]:
# Cleaning Team Names
nba.columns = [header.lower() for header in nba.columns]

# Changing team abbreviations to match modern versions
nba = nba.replace({'am':{
    'CHO': 'CHA',
    'CHH': 'CHA',
    'BRK': 'BKN',
    'NJN': 'BKN',
    'VAN': 'MEM',
    'PHO': 'PHX',
    'NOH': 'NOP',
    'NOK': 'NOP',
    'SEA': 'OKC',
}})
print(f"Number of NBA teams (including 'TOT'): {len(nba.tm.unique())}")

Number of NBA teams (including 'TOT'): 38


In the above cell, we changed team abbreivations to match modern versions. Thus, there will be 31 values corresponding to team, representing each of the 30 NBA franchise present in our dataset (no defunct teams are apart of our dataset) and a 31st value 'TOT', which means the player played for multiple teams that season.

In [13]:
# Adding percentage of games started
nba.insert(6,'gs%',(nba.gs/nba.g).round(decimals=3))

In [14]:
# Addressing player names with accents
names = nba.player.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
nba.player = names

In the above cell, we standardized player names, particularly those with accents, which were handled differently by the differently by Stathead and PBP.

In [15]:
# Removing duplicate rows with players playing for multiple teams in one season
# Keeping row of aggregate stats for that player for that season
nba = nba.drop_duplicates(subset = ['player','season'])

Players who switched teams in the middle of a season had entires in our Stathead/Basketball Reference DataFrame for both their overall season numbers, and their splits for each team they played for. This cell drops the extra entries, which correspond to their team splits, and keeps just their season-long numbers.

In [16]:
# 70% mins cutoff
mins70 = list(nba.groupby('season').mp.quantile(q=0.3))
all_bl = [False]*len(nba)
year = 2001
for i in np.arange(len(mins70)):
    bl = np.logical_and(nba.season==year , nba.mp > mins70[i])
    all_bl = np.logical_or(all_bl, bl)
    year += 1

In [17]:
# Assigning final rows of df
nba_filtered = nba.loc[all_bl]

In the above two cells, we remove all players who were not above the 30th percentile in minutes played in a particular season. This is due to a few reasons. The first, and primary, reason is because some of our data was not available for many players whose minutes played placed them below the cutoff. Additionally, per-possession and existing advanced statistics are subject to extreme noise when examined for players with lower amounts of minutes played, as these small sample sizes are not representative of a player’s true ability and are often strongly influenced by external factors.

In [18]:
# Change datatypes of certain columns to match their true type
nba_filtered = nba_filtered.astype({'season': int, 'age': int, 'g': int, 'gs': int, 'mp': int, 'fg': int, 'fga': int,
                                  '3p': int, '3pa': int, '2p': int, '2pa': int, 'ft': int, 'fta': int, 'orb': int,
                                  'drb': int, 'trb': int, 'ast': int, 'stl': int, 'blk': int, 'tov': int, 'pf': int,
                                  'pts': int})
nba_filtered.head()

,season,player,age,tm,g,gs,gs%,mp,per,ts%,...,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts
2,2001,Shareef Abdur-Rahim,24,VAN,81,81,1.000,3241,19.1,0.549,...,0.834,175,560,735,250,90,77,231,238,1663
4,2001,Courtney Alexander,23,TOT,65,24,0.369,1382,11.6,0.484,...,0.820,42,101,143,62,45,5,75,139,618
7,2001,Ray Allen,25,MIL,82,82,1.000,3129,22.9,0.610,...,0.888,101,327,428,374,124,20,204,192,1806
9,2001,John Amaechi,30,ORL,82,36,0.439,1710,8.7,0.455,...,0.631,77,191,268,74,28,29,124,175,650
10,2001,Derek Anderson,26,SAS,82,82,1.000,2859,17.1,0.542,...,0.851,75,288,363,301,120,14,165,188,1269


In the above cell, we changed the datatype of certain columns to reflect their true type. When our data was read in, these columns all had the datatype float64, which is nonsensical.

In [19]:
# Drop some irrelevant, unnecessary, and/or redundant columns from PBP data
pbp_score = pbp_score.drop(['Unnamed: 0', 'TeamAbbreviation', 'GamesPlayed', 'Minutes', 'Points','FG2M', 'FG2A',
                            'Fg2Pct', 'FG3M', 'FG3A', 'Fg3Pct', 'FG3APct','EfgPct', 'TsPct','Usage'], axis = 1)
pbp_assist = pbp_assist.drop(['Unnamed: 0', 'TeamAbbreviation', 'GamesPlayed', 'Minutes', 'Assists'], axis = 1)

Here, we dropped columns which were either irrelevant, unnecessary, or redundant from our PBP DataFrames.

In [20]:
# Tidying up columns in PBP data
pbp_score.columns = [header.lower() for header in pbp_score.columns]
pbp_assist.columns = [header.lower() for header in pbp_assist.columns]

In [21]:
# Rename name to player for pbp
pbp_score = pbp_score.rename(columns = {'name' : 'player'})
pbp_assist = pbp_assist.rename(columns = {'name' : 'player'})

# Merge 2 pbp files
pbp = pbp_score.merge(pbp_assist)

In the prior cell, we changed the name of the 'name' column to 'player' in our PBP DataFrames to match the corresponding column name in the Stathead/Basketball Reference DataFrame. Additionally, we merged the two PBP DataFrames to make one PBP DataFrame containing both the scoring and assist data.

In [22]:
# Merge dfs from the 2 different datasets to create single df
nba_1 = nba_filtered.merge(pbp, how = 'inner', on = ['player', 'season'])

# Adjust to modern team abbreviations
nba_1 = nba_1.replace({'tm':{
    'CHO': 'CHA',
    'CHH': 'CHA',
    'BRK': 'BKN',
    'NJN': 'BKN',
    'VAN': 'MEM',
    'PHO': 'PHX',
    'NOH': 'NOP',
    'NOK': 'NOP',
    'SEA': 'OKC',
}})

Here, we merged our Stathead/Basketball Reference and PBP DataFrames to one, comprehensive DataFrame.

In [23]:
# Adjusting player names of nba player position df
pos_names = nba_pos.player.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
nba_pos.player = pos_names
# Assigning position to corresponding player
nba = nba_1.merge(nba_pos, on = 'player', how = 'left')
# Renaming team name column to 'team'
nba = nba.rename(columns = {'tm': 'team'})

Earlier, we removed the column containing player position in our Stathead/Basketball Reference 2000-2017 DataFrame because it was not present in any of the other datasheets we were going to be working with. However, because having the position for each player could enhance our analysis, we added it to our final DataFrame in the above cell.

In [24]:
# Manually adding position of 1 player, Omer Asik, whose position was missing from the csv
omer_i = list(nba.loc[nba.pos.isnull()].index)
nba.loc[omer_i,'pos'] = "C"

Our position DataFrame had one missing value: it did not include the position for Omer Asik. In the above cell, we manually added it in.

In [25]:
nba.columns

Index(['season', 'player', 'age', 'team', 'g', 'gs', 'gs%', 'mp', 'per', 'ts%',
       '3par', 'ftr', 'orb%', 'drb%', 'trb%', 'ast%', 'stl%', 'blk%', 'tov%',
       'usg%', 'ows', 'dws', 'ws', 'ws/48', 'obpm', 'dbpm', 'bpm', 'vorp',
       'fg', 'fga', 'fg%', '3p', '3pa', '3p%', '2p', '2pa', '2p%', 'efg%',
       'ft', 'fta', 'ft%', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov',
       'pf', 'pts', 'offposs', 'nonheavefg3pct', 'ftpoints', 'ptsassisted2s',
       'ptsunassisted2s', 'ptsassisted3s', 'ptsunassisted3s', 'assisted2spct',
       'nonputbacksassisted2spct', 'assisted3spct', 'shotqualityavg',
       'ptsputbacks', 'fg2ablocked', 'fg2apctblocked', 'fg3ablocked',
       'fg3apctblocked', 'assistpoints', 'twoptassists', 'threeptassists',
       'atrimassists', 'shortmidrangeassists', 'longmidrangeassists',
       'corner3assists', 'arc3assists', 'pos'],
      dtype='object')

In [26]:
# Read in csv containing league average team shooting stats for the 2000-2001 to the 2020-2021 season.
lg_avg_shooting = pd.read_csv('NBA Team Shooting Lg Avgs 2001-2021.csv')
lg_avg_shooting.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,% of FGA by Distance,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,% of FG Ast'd,Unnamed: 18,Dunks,Unnamed: 20,Corner,Unnamed: 22,Heaves,Unnamed: 24
0,Season,G,MP,FG%,Dist.,2P,0-3 ft,10-Mar,16-Oct,16-3pt,...,16-3pt,3P,2P,3P,%FGA,Md.,%3PA,3P%,Att.,Md.
1,2021,72,17381,0.466,14.1,0.608,0.254,0.186,0.096,0.072,...,0.409,0.367,0.502,0.826,0.054,303,0.222,0.395,10.5,0.3
2,2020,71,17061,0.46,13.9,0.616,0.282,0.165,0.093,0.077,...,0.404,0.358,0.504,0.815,0.059,330,0.211,0.39,13.9,0.3
3,2019,82,19818,0.461,13.5,0.641,0.292,0.162,0.096,0.092,...,0.401,0.355,0.513,0.823,0.058,380,0.214,0.383,15.7,0.5
4,2018,82,19795,0.46,13.2,0.663,0.281,0.156,0.106,0.12,...,0.4,0.362,0.499,0.83,0.053,331,0.208,0.394,17.5,0.2


In the above cell, we imported league average team shooting statistics for all seasons from 2000-2001 to 2020-2021. To get this data in this form, we went to Basketball Reference's season summary page for every season in the previously mentioned range, and copied the column values for the observation "League Average" in the "Shooting Stats" table. As can be seen from the above output, this data is in desperate need of cleaning.

In [27]:
# Removing the non-sense column names and replacing them with the first observation in 
# the df, which corresponds to the true column names.
new_col_names = lg_avg_shooting.loc[0,:]
lg_avg_shooting.columns = new_col_names
# Removing the first row in the df, which does not correspond to an actual observation/season.
lg_avg_shooting = lg_avg_shooting.loc[1:,:]
lg_avg_shooting.head()

,Season,G,MP,FG%,Dist.,2P,0-3 ft,10-Mar,16-Oct,16-3pt,...,16-3pt,3P,2P,3P,%FGA,Md.,%3PA,3P%,Att.,Md.
1,2021,72,17381,0.466,14.1,0.608,0.254,0.186,0.096,0.072,...,0.409,0.367,0.502,0.826,0.054,303,0.222,0.395,10.5,0.3
2,2020,71,17061,0.46,13.9,0.616,0.282,0.165,0.093,0.077,...,0.404,0.358,0.504,0.815,0.059,330,0.211,0.39,13.9,0.3
3,2019,82,19818,0.461,13.5,0.641,0.292,0.162,0.096,0.092,...,0.401,0.355,0.513,0.823,0.058,380,0.214,0.383,15.7,0.5
4,2018,82,19795,0.46,13.2,0.663,0.281,0.156,0.106,0.12,...,0.4,0.362,0.499,0.83,0.053,331,0.208,0.394,17.5,0.2
5,2017,82,19815,0.457,13.3,0.684,0.29,0.152,0.099,0.142,...,0.403,0.358,0.495,0.835,0.051,332,0.228,0.388,16.8,0.3


In [28]:
# First, tidy up league_avg_shooting column names, and add in information about each variable lost by 
# changing the original column names.
lg_avg_shooting = lg_avg_shooting.rename(columns = {'Dist.': 'dist (ft)', '2P': 'pct_FGA_2P', '0-3 ft': 
                                                    'pct_FGA_0to3_ft', '10-Mar': 'pct_FGA_3to10_ft', 
                                                    '16-Oct': 'pct_FGA_10to16_ft', '16-3pt': 'pct_FGA_16ftto3PT',
                                                    '3P': 'pct_FGA_3PT', '2P': 'pct_2PM', '0-3': 'pct_FGM_0to3_ft', 
                                                    '10-Mar': 'pct_FGM_3to10_ft', '16-Oct': 'pct_FGM_10to16_ft',
                                                    '16-3pt': 'pct_FGM_16ftto3PT', '3P': 'pct_3PM', 
                                                    '2P': 'pct_2PM_assisted', '3P': 'pct_3PM_assisted', 
                                                    '%FGA': 'pct_FGA_dunks', 'Md.': 'dunks_made', 
                                                    '%3PA': 'pct_3PA_corner', '3P%': 'pct_3PM_corner', 
                                                    'Att.': 'heaves_att', 'Md.': 'heaves_made'})
# Because some columns have the same name, some of our columns did not receive their desired name, 
# receiving their counterpart's name as well. This occur for columns with original names '2P','3P', '10-Mar', '16-Oct'
# '16-3pt', and 'Md.'.
duplicates_idx = lg_avg_shooting.columns.duplicated(keep = 'last')
print(lg_avg_shooting.columns[duplicates_idx])
# Examining the printed output from the above line, we notice that the first instance of each duplicate must be 
# the one needed to be changed. We will change them directly to avoid the same issue.
shooting_cols = list(lg_avg_shooting.columns)
shooting_cols[5] = 'pct_FGA_2P'
shooting_cols[7] = 'pct_FGA_3to10_ft'
shooting_cols[8] = 'pct_FGA_10to16_ft'
shooting_cols[9] = 'pct_FGA_16ftto3PT'
shooting_cols[10] = 'pct_FGA_3PT'
shooting_cols[11] = 'pct_2PM'
shooting_cols[16] = 'pct_3PM'
shooting_cols[20] = 'dunks_made'
lg_avg_shooting.columns = shooting_cols

Index(['pct_2PM_assisted', 'pct_FGM_3to10_ft', 'pct_FGM_10to16_ft',
       'pct_FGM_16ftto3PT', 'pct_3PM_assisted', 'pct_2PM_assisted',
       'pct_3PM_assisted', 'heaves_made'],
      dtype='object', name=0)


In [29]:
print(lg_avg_shooting.columns)

Index(['Season', 'G', 'MP', 'FG%', 'dist (ft)', 'pct_FGA_2P',
       'pct_FGA_0to3_ft', 'pct_FGA_3to10_ft', 'pct_FGA_10to16_ft',
       'pct_FGA_16ftto3PT', 'pct_FGA_3PT', 'pct_2PM', 'pct_FGM_0to3_ft',
       'pct_FGM_3to10_ft', 'pct_FGM_10to16_ft', 'pct_FGM_16ftto3PT', 'pct_3PM',
       'pct_2PM_assisted', 'pct_3PM_assisted', 'pct_FGA_dunks', 'dunks_made',
       'pct_3PA_corner', 'pct_3PM_corner', 'heaves_att', 'heaves_made'],
      dtype='object')


In the above cells, we begin tidying up league_avg_shooting's column names. We did this before dropping the irrelevant or unnecessary columns so it would be clear what exactly we were dropping.

In [30]:
# Now, dropping the columns that we are certain are irrelevant or unnecessary to our analysis.
lg_avg_shooting = lg_avg_shooting.drop(['G', 'MP', 'FG%', 'dist (ft)', 'pct_FGA_0to3_ft','pct_FGA_16ftto3PT', 
                                        'pct_FGA_3PT', 'pct_FGA_dunks', 'dunks_made','heaves_att', 'heaves_made'],
                                       axis = 1)
# Making all columns lowercase...
lg_avg_shooting.columns = [col_name.lower() for col_name in lg_avg_shooting.columns]
print(lg_avg_shooting.columns)

Index(['season', 'pct_fga_2p', 'pct_fga_3to10_ft', 'pct_fga_10to16_ft',
       'pct_2pm', 'pct_fgm_0to3_ft', 'pct_fgm_3to10_ft', 'pct_fgm_10to16_ft',
       'pct_fgm_16ftto3pt', 'pct_3pm', 'pct_2pm_assisted', 'pct_3pm_assisted',
       'pct_3pa_corner', 'pct_3pm_corner'],
      dtype='object')


In the above cell, we dropped irrelevant, unnecessary columns from our league average team shooting stats DataFrame and made all of our remaining columns lowercase.

In [31]:
# Finally, let's examine the datatype of each column in league_avg_shooting...
print(lg_avg_shooting.dtypes)

season               object
pct_fga_2p           object
pct_fga_3to10_ft     object
pct_fga_10to16_ft    object
pct_2pm              object
pct_fgm_0to3_ft      object
pct_fgm_3to10_ft     object
pct_fgm_10to16_ft    object
pct_fgm_16ftto3pt    object
pct_3pm              object
pct_2pm_assisted     object
pct_3pm_assisted     object
pct_3pa_corner       object
pct_3pm_corner       object
dtype: object


In [32]:
# Looking at the output directly above this cell, we can see that none of our variables' current datatypes 
# are the same as their true datatypes. Let's fix this:
lg_avg_shooting = lg_avg_shooting.astype({'season': int, 'pct_2pm': float, 'pct_fgm_0to3_ft': float, 
                                           'pct_fgm_3to10_ft': float, 'pct_fgm_10to16_ft': float, 
                                           'pct_fgm_16ftto3pt': float, 'pct_3pm': float, 'pct_2pm_assisted': float, 
                                           'pct_3pm_assisted': float, 'pct_3pm_corner': float, 
                                           'pct_3pa_corner': float, 'pct_fga_2p': float, 'pct_fga_3to10_ft': float,
                                           'pct_fga_10to16_ft': float})
print(lg_avg_shooting.dtypes)

season                 int64
pct_fga_2p           float64
pct_fga_3to10_ft     float64
pct_fga_10to16_ft    float64
pct_2pm              float64
pct_fgm_0to3_ft      float64
pct_fgm_3to10_ft     float64
pct_fgm_10to16_ft    float64
pct_fgm_16ftto3pt    float64
pct_3pm              float64
pct_2pm_assisted     float64
pct_3pm_assisted     float64
pct_3pa_corner       float64
pct_3pm_corner       float64
dtype: object


In the above cells, we altered the datatypes of the columns in our league average team shooting stats DataFrame to match their true datatypes.

In [33]:
# Read in csv containing data on NBA team statistics for seasons from 2000 to 2021.
teamstats_raw = pd.read_csv('teamstats_raw.csv')

In the above cell, we imported team statistics from each team in the league for all seasons from 2000-2001 to 2020-2021. To get this data in this form, we exported season-long team statistics from the season summary page of Basketball Reference. We then concatenated each season's data into a single csv file for ease of use.

In [34]:
# Cleaning
teamstats_raw.reset_index()
teamstats_raw.columns = [header.lower() for header in teamstats_raw.columns]
keep_cols = ['rk', 'team', 'age', 'w', 'l', 'pw', 'pl', 'mov', 'sos', 'srs', 'ortg',
       'drtg', 'nrtg', 'pace', 'ftr', '3par', 'ts%', 'efg%',
       'tov%', 'orb%', 'ft/fga', 'efg%.1', 'tov%.1', 'drb%',
       'ft/fga.1', 'attend.', 'attend./g', 'season']
teamstats_raw = teamstats_raw.loc[teamstats_raw.team != 'League Average',keep_cols]

teamstats_raw = teamstats_raw.rename(columns={
    'rk' : 'rank',
    'w' : 'wins',
    'l' : 'losses'})

# Team Names
teamstats_raw.team = [team.replace('*','') for team in teamstats_raw.team]
teamstats_raw = teamstats_raw.replace({'team':{
    'San Antonio Spurs':'SAS','Sacramento Kings':'SAC','Utah Jazz':'UTA','Philadelphia 76ers':'PHI',
    'Portland Trail Blazers':'POR','Dallas Mavericks':'DAL','Milwaukee Bucks':'MIL',
    'Los Angeles Lakers':'LAL','New York Knicks':'NYK','Miami Heat':'MIA','Toronto Raptors':'TOR',
    'Houston Rockets':'HOU','Charlotte Hornets':'CHA','Phoenix Suns':'PHX',
    'Minnesota Timberwolves':'MIN','Orlando Magic':'ORL','Seattle SuperSonics':'OKC',
    'Indiana Pacers':'IND','Detroit Pistons':'DET','Boston Celtics':'BOS','Denver Nuggets':'DEN',
    'Los Angeles Clippers':'LAC','Cleveland Cavaliers':'CLE','New Jersey Nets':'BKN',
    'Atlanta Hawks':'ATL','Vancouver Grizzlies':'MEM','Washington Wizards':'WAS',
    'Golden State Warriors':'GSW','Chicago Bulls':'CHI','Memphis Grizzlies':'MEM',
    'New Orleans Hornets':'NOP','Charlotte Bobcats':'CHA','New Orleans/Oklahoma City Hornets':'NOP',
    'Oklahoma City Thunder':'OKC','Brooklyn Nets':'BKN','New Orleans Pelicans':'NOP'
}})
teamstats = teamstats_raw

In the cell above, we remove certain unnecessary columns and also the rows for the league average values of a given season. We then update each team name to its modern day franchise's abbreviation so that this data is compatible with the other datasets we are using.

In [35]:
# Converting values to appropriate data types
teamstats = teamstats.astype({
    'rank' : int, 'age' : int, 'wins' : int, 'losses' : int})

In the above cell, we altered the datatypes of the columns in our DataFrame to match their true datatypes.

In [36]:
# Reading in a csv containing league average team statistics by season...
lg_avg_teamstats = pd.read_csv('NBA Team Lg Avg Data 1949-2021.csv')
lg_avg_teamstats.head()

,Unnamed: 0,Unnamed: 1,Per Game,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Shooting,Unnamed: 19,Unnamed: 20,Advanced,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,Season,G,MP,FG,FGA,3P,3PA,FT,FTA,ORB,...,PTS,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg
1,2020-21,1080,241.4,41.2,88.4,12.7,34.6,17,21.8,9.8,...,112.1,0.466,0.367,0.778,99.2,0.538,12.4,22.2,0.192,112.3
2,2019-20,1059,241.8,40.9,88.8,12.2,34.1,17.9,23.1,10.1,...,111.8,0.46,0.358,0.773,100.3,0.529,12.8,22.5,0.201,110.6
3,2018-19,1230,241.6,41.1,89.2,11.4,32,17.7,23.1,10.3,...,111.2,0.461,0.355,0.766,100,0.524,12.4,22.9,0.198,110.4
4,2017-18,1230,241.4,39.6,86.1,10.5,29,16.6,21.7,9.7,...,106.3,0.46,0.362,0.767,97.3,0.521,13,22.3,0.193,108.6


In the above cell, we imported league average team statistics for all seasons from 1949-1950 to 2020-2021. This data is from Basketball Reference as well.

In [37]:
# Removing the non-sense column names and replacing them with the first observation in 
# the df, which corresponds to the true column names.
new_col_names2 = lg_avg_teamstats.loc[0,:]
lg_avg_teamstats.columns = new_col_names2
# Removing the first row in the df, which does not correspond to an actual observation/season.
lg_avg_teamstats = lg_avg_teamstats.loc[1:,:]
lg_avg_teamstats.head()

,Season,G,MP,FG,FGA,3P,3PA,FT,FTA,ORB,...,PTS,FG%,3P%,FT%,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg
1,2020-21,1080,241.4,41.2,88.4,12.7,34.6,17,21.8,9.8,...,112.1,0.466,0.367,0.778,99.2,0.538,12.4,22.2,0.192,112.3
2,2019-20,1059,241.8,40.9,88.8,12.2,34.1,17.9,23.1,10.1,...,111.8,0.46,0.358,0.773,100.3,0.529,12.8,22.5,0.201,110.6
3,2018-19,1230,241.6,41.1,89.2,11.4,32,17.7,23.1,10.3,...,111.2,0.461,0.355,0.766,100,0.524,12.4,22.9,0.198,110.4
4,2017-18,1230,241.4,39.6,86.1,10.5,29,16.6,21.7,9.7,...,106.3,0.46,0.362,0.767,97.3,0.521,13,22.3,0.193,108.6
5,2016-17,1230,241.6,39,85.4,9.7,27,17.8,23.1,10.1,...,105.6,0.457,0.358,0.772,96.4,0.514,12.7,23.3,0.209,108.8


In the above cell, we started tidying up lg_avg_teamstats, fixing the columns so that they are labelled as they should be. And we removed the DataFrame's first row, which had contained the true column labels.

In [38]:
lg_avg_teamstats = lg_avg_teamstats.reset_index()
# Removing all unneccesary observations (seasons not present in our dataset) and columns...
lg_avg_teamstats = lg_avg_teamstats.iloc[0:21, :]
# Removing all columns we know to be unnecessary or redundant from our df lg_avg_teamstats...
lg_avg_teamstats = lg_avg_teamstats.drop(columns = ['index', 'G', 'MP', 'ORB', 'DRB', 'STL', 
                                                    'BLK', 'TRB', 'FT%', 'PF', 'ORB%'] )
lg_avg_teamstats

,Season,FG,FGA,3P,3PA,FT,FTA,AST,TOV,PTS,FG%,3P%,Pace,eFG%,TOV%,FT/FGA,ORtg
0,2020-21,41.2,88.4,12.7,34.6,17,21.8,24.8,13.8,112.1,0.466,0.367,99.2,0.538,12.4,0.192,112.3
1,2019-20,40.9,88.8,12.2,34.1,17.9,23.1,24.4,14.5,111.8,0.46,0.358,100.3,0.529,12.8,0.201,110.6
2,2018-19,41.1,89.2,11.4,32,17.7,23.1,24.6,14.1,111.2,0.461,0.355,100,0.524,12.4,0.198,110.4
3,2017-18,39.6,86.1,10.5,29,16.6,21.7,23.2,14.3,106.3,0.46,0.362,97.3,0.521,13,0.193,108.6
4,2016-17,39,85.4,9.7,27,17.8,23.1,22.6,14,105.6,0.457,0.358,96.4,0.514,12.7,0.209,108.8
5,2015-16,38.2,84.6,8.5,24.1,17.7,23.4,22.3,14.4,102.7,0.452,0.354,95.8,0.502,13.2,0.209,106.4
6,2014-15,37.5,83.6,7.8,22.4,17.1,22.8,22,14.4,100,0.449,0.35,93.9,0.496,13.3,0.205,105.6
7,2013-14,37.7,83,7.7,21.5,17.8,23.6,22,14.6,101,0.454,0.36,93.9,0.501,13.6,0.215,106.6
8,2012-13,37.1,82,7.2,20,16.7,22.2,22.1,14.6,98.1,0.453,0.359,92,0.496,13.7,0.204,105.8
9,2011-12,36.5,81.4,6.4,18.4,16.9,22.5,21,14.6,96.3,0.448,0.349,91.3,0.487,13.8,0.208,104.6


In the above cell, we dropped observations from prior to the 2000-2001 season, as they go beyond the scope of our already exisitng dataset. And, we dropped unnecessary or redundant columns.

In [39]:
# Converting Season column values to be the same as in our other dfs
for i in np.arange(len(lg_avg_teamstats)):
    lg_avg_teamstats.loc[i,'Season'] = int(lg_avg_teamstats.loc[i,'Season'][-2:])+2000
# Making all columns lowercase
lg_avg_teamstats.columns = [j.lower() for j in lg_avg_teamstats.columns]
lg_avg_teamstats

,season,fg,fga,3p,3pa,ft,fta,ast,tov,pts,fg%,3p%,pace,efg%,tov%,ft/fga,ortg
0,2021,41.2,88.4,12.7,34.6,17,21.8,24.8,13.8,112.1,0.466,0.367,99.2,0.538,12.4,0.192,112.3
1,2020,40.9,88.8,12.2,34.1,17.9,23.1,24.4,14.5,111.8,0.46,0.358,100.3,0.529,12.8,0.201,110.6
2,2019,41.1,89.2,11.4,32,17.7,23.1,24.6,14.1,111.2,0.461,0.355,100,0.524,12.4,0.198,110.4
3,2018,39.6,86.1,10.5,29,16.6,21.7,23.2,14.3,106.3,0.46,0.362,97.3,0.521,13,0.193,108.6
4,2017,39,85.4,9.7,27,17.8,23.1,22.6,14,105.6,0.457,0.358,96.4,0.514,12.7,0.209,108.8
5,2016,38.2,84.6,8.5,24.1,17.7,23.4,22.3,14.4,102.7,0.452,0.354,95.8,0.502,13.2,0.209,106.4
6,2015,37.5,83.6,7.8,22.4,17.1,22.8,22,14.4,100,0.449,0.35,93.9,0.496,13.3,0.205,105.6
7,2014,37.7,83,7.7,21.5,17.8,23.6,22,14.6,101,0.454,0.36,93.9,0.501,13.6,0.215,106.6
8,2013,37.1,82,7.2,20,16.7,22.2,22.1,14.6,98.1,0.453,0.359,92,0.496,13.7,0.204,105.8
9,2012,36.5,81.4,6.4,18.4,16.9,22.5,21,14.6,96.3,0.448,0.349,91.3,0.487,13.8,0.208,104.6


In the above cell, we finished tidying up lg_avg_teamstats' columns and altered the values in the season column such that they measured those in the other DataFrames we have cleaned thus far.

In [40]:
# Finally, let's examine the datatypes of the columns in our lg_avg_teamstats df.
print(lg_avg_teamstats.dtypes)

season    object
fg        object
fga       object
3p        object
3pa       object
ft        object
fta       object
ast       object
tov       object
pts       object
fg%       object
3p%       object
pace      object
efg%      object
tov%      object
ft/fga    object
ortg      object
dtype: object


In [41]:
# Looking at the output directly above this cell, we can see that none of our variables' current datatypes 
# are the same as their true datatypes. Let's fix this:
lg_avg_teamstats = lg_avg_teamstats.astype({'season': int, 'fg': float, 'fga': float, '3p': float, '3pa': float, 
                                           'ft': float, 'fta': float, 'ast': float, 'tov': float, 'pts': float, 
                                           'fg%': float, '3p%': float, 'pace': float,'efg%': float, 'tov%': float, 
                                           'ft/fga': float, 'ortg': float})
print(lg_avg_teamstats.dtypes)

season      int64
fg        float64
fga       float64
3p        float64
3pa       float64
ft        float64
fta       float64
ast       float64
tov       float64
pts       float64
fg%       float64
3p%       float64
pace      float64
efg%      float64
tov%      float64
ft/fga    float64
ortg      float64
dtype: object


In the above cell, we changed the datatype of the columns in our lg_avg_teamstats DataFrame so they match their true types.

In [42]:
# Writing our DataFrames to csv files...
nba.to_csv('nba.csv', index = False)
lg_avg_shooting.to_csv('lg_avg_shooting.csv', index = False)
teamstats.to_csv('teamstats.csv', index = False)
lg_avg_teamstats.to_csv('lg_avg_teamstats.csv', index = False)